In [ ]:
%pip install bardapi
%pip install openai
%pip install -q -U google-generativeai
%pip install hugchat
%pip install pandas

In [2]:
import numpy as np
import pandas as pd
import random
import os
import ast
# from google.colab import userdata
# import google.generativeai as genai
# from hugchat import hugchat
# from hugchat.login import Login
import time

In [ ]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [4]:
import glob
import os
# if path.exists('./drive/MyDrive/SemEval2024/Task9/SP_new_test.npy')
# ./drive/MyDrive/SemEval2024/Task9/Round_{round}.csv
folder_path = '/home/jovyan/work'
pattern = 'Round_*.csv'

# Find all files matching the pattern
files = glob.glob(os.path.join(folder_path, pattern))

if files:
    # Extract round numbers from the file names
    rounds = [int(file.split('_')[1].split('.')[0]) for file in files]

    # Find the file with the highest round
    max_round_file = max(zip(rounds, files), key=lambda x: x[0])[1]

    # Read the DataFrame from the file with the highest round
    df = pd.read_csv(max_round_file)

    # Display the DataFrame or perform other operations
    # print(df)
    print("Read from existing DataFrame")
else:
    SP_eval_data = np.load('/home/jovyan/work/SP_new_test.npy', allow_pickle=True)
    df = pd.DataFrame(list(SP_eval_data))
    print("Read from original Dataset")


Read from original Dataset


In [5]:
df_columns = df.columns
df_columns
num_rounds = 4

In [ ]:
model_names = ['ChatGPT', 'Gemini', 'Mistral']

import openai

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_ChatGPT_answer(row, round):
    choice_list = None
    if isinstance(row['choice_list'], str):
        choice_list = ast.literal_eval(row['choice_list'])
    elif isinstance(row['choice_list'], list):
        choice_list = row['choice_list']
    prompt = f""""""
    if round == 1:
#         prompt += f"""
# you are facing with a riddle or tricky sentence. let's work this out in a step by step way to be sure we have the right answer. find a way to solve it based on options and tell your reason. please before answering the question see options want what? :
# Q: {row['question']}
# 0) {choice_list[0]}
# 1) {choice_list[1]}
# 2) {choice_list[2]}
# 3) {choice_list[3]}

# write your answer in the following format:
# Answer: <option_number(0-based)>
# Confidence: <confidence of you answer from 0 to 100>%
# Reason: <reason>
# """
        prompt += f"""
let's think step by step. among all possible answer choose best one.
Riddle: {row['question']}
option 0) {choice_list[0]}
option 1) {choice_list[1]}
option 2) {choice_list[2]}
option 3) {choice_list[3]}

write your answer in the following format:
Answer: <option_number(0-based)>
Confidence: <confidence of you answer from 0 to 100>%
Reason: <reason>
"""
    else:
#         prompt += f"""
# you are facing with a riddle or tricky sentence. let's work this out in a step by step way to be sure we have the right answer. find a way to solve it based on options and tell your reason. please before answering the question see options want what? :
# Q: {row['question']}
# 0) {choice_list[0]}
# 1) {choice_list[1]}
# 2) {choice_list[2]}
# 3) {choice_list[3]}

# these are answers of other agents (their answer is also in 0-based format):
# """
        prompt += f"""
let's think step by step. among all possible answer choose best one.
Riddle: {row['question']}
option 0) {choice_list[0]}
option 1) {choice_list[1]}
option 2) {choice_list[2]}
option 3) {choice_list[3]}

these are answers of other agents (their answer is also in 0-based format):
"""
        tmp = [f"some agent answer: {row[f'Round_{round-1}_Model_{model_name}']}" for idx, model_name in enumerate(model_names)]
        s = '\n'.join(tmp)
        prompt += s

        prompt += """
Check if the answer of other agents are logical or not and if you think they don't have logical reason, write your logical reason. Write your answer to the tricky question in the following format:
Answer: <option_number (0-based)>
Confidence: <confidence of you answer from 0 to 100>%
Reason: <reason>
"""
    print(prompt)
    print(50*"*")
    response = get_completion(prompt)
    return response






def get_gemini_answer(row, round):
    raise Exception("exception")
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-pro')
    choice_list = None
    if isinstance(row['choice_list'], str):
        choice_list = ast.literal_eval(row['choice_list'])
    elif isinstance(row['choice_list'], list):
        choice_list = row['choice_list']

    prompt = f""""""
    if round == 1:
        prompt += f"""
let's think step by step. among all possible answer choose best one.
Riddle: {row['question']}
option 0) {choice_list[0]}
option 1) {choice_list[1]}
option 2) {choice_list[2]}
option 3) {choice_list[3]}

write your answer in the following format:
Answer: <option_number(0-based)>
Confidence: <confidence of you answer from 0 to 100>%
Reason: <reason>
"""
    else:
        prompt += f"""
let's think step by step. among all possible answer choose best one.
Riddle: {row['question']}
option 0) {choice_list[0]}
option 1) {choice_list[1]}
option 2) {choice_list[2]}
option 3) {choice_list[3]}

these are answers of other agents (their answer is also in 0-based format):
"""
        tmp = [f"some agent answer: {row[f'Round_{round-1}_Model_{model_name}']}" for idx, model_name in enumerate(model_names)]
        s = '\n'.join(tmp)
        prompt += s

        prompt += """
Check if the answer of other agents are logical or not and if you think they don't have logical reason, write your logical reason. Write your answer to the tricky question in the following format:
Answer: <option_number (0-based)>
Confidence: <confidence of you answer from 0 to 100>%
Reason: <reason>
"""
    response = model.generate_content(prompt)

    return response.text



def get_Mistral_answer(row, round):
    LOGIN_EMAIL=userdata.get('LOGIN_EMAIL')
    LOGIN_PASS=userdata.get('LOGIN_PASS')
    # sign = Login(email="", passwd="")
    sign = Login(email=LOGIN_EMAIL, passwd=LOGIN_PASS)
    cookies = sign.login()

    # Save cookies to the local directory
    cookie_path_dir = "./cookies_snapshot"
    sign.saveCookiesToDir(cookie_path_dir)

    # Create a ChatBot
    chatbot = hugchat.ChatBot(cookies=cookies.get_dict())  # or cookie_path="usercookies/<email>.json"

    # Create a new conversation
    id = chatbot.new_conversation()
    chatbot.change_conversation(id)



    choice_list = None
    if isinstance(row['choice_list'], str):
        choice_list = ast.literal_eval(row['choice_list'])
    elif isinstance(row['choice_list'], list):
        choice_list = row['choice_list']

    prompt = f""""""
    if round == 1:
        prompt += f"""
let's think step by step. among all possible answer choose best one.
Riddle: {row['question']}
option 0) {choice_list[0]}
option 1) {choice_list[1]}
option 2) {choice_list[2]}
option 3) {choice_list[3]}

write your answer in the following format:
Answer: <option_number(0-based)>
Confidence: <confidence of you answer from 0 to 100>%
Reason: <reason>
"""
    else:
        prompt += f"""
let's think step by step. among all possible answer choose best one.
Riddle: {row['question']}
option 0) {choice_list[0]}
option 1) {choice_list[1]}
option 2) {choice_list[2]}
option 3) {choice_list[3]}

these are answers of other agents (their answer is also in 0-based format):
"""
        tmp = [f"some agent answer: {row[f'Round_{round-1}_Model_{model_name}']}" for idx, model_name in enumerate(model_names)]
        s = '\n'.join(tmp)
        prompt += s

        prompt += """
Check if the answer of other agents are logical or not and if you think they don't have logical reason, write your logical reason. Write your answer to the tricky question in the following format:
Answer: <option_number (0-based)>
Confidence: <confidence of you answer from 0 to 100>%
Reason: <reason>
"""

    # non stream response
    response = chatbot.query(prompt)['text']
    # print(query_result) # or query_result.text or query_result["text"]
    return response




get_model_answer = [get_ChatGPT_answer, get_gemini_answer, get_Mistral_answer]

In [ ]:
import re
round_models = {it:[] for it in model_names}
for model in model_names:
    for col in df_columns:
        if col.startswith('Round_') and f'_Model_{model}' in col:
            parts = col.split('_')
            round_num = int(parts[1])
            model_name = parts[3]
            round_models[model_name] = sorted(round_models.get(model_name, []) + [round_num])

In [ ]:
round_models

In [ ]:
saved = []
for k in round_models.keys():
    if len(round_models[k]) > 0:
        if df[f'Round_{max(round_models[k])}_Model_{k}'].notnull().all():
            saved.append(max(round_models[k]))
        else:
            saved.append(max(round_models[k])-1)
    else:
        saved.append(0)

last_saved_round = min(saved)

In [ ]:
first_unsaved_round = last_saved_round + 1
first_unsaved_round

In [ ]:
def get_value(model_name, row, current_round):
    response = get_model_answer[model_names.index(model_name)](row, current_round)
    return response

In [ ]:
from tqdm import tqdm
for current_round in range(first_unsaved_round, num_rounds + 1):
    for model_name in model_names:
        if f'Round_{current_round}_Model_{model_name}' not in df.columns:
            df[f'Round_{current_round}_Model_{model_name}'] = pd.Series()
        for index, row in tqdm(df.iterrows(), total = len(df), desc = f"{model_name}"):
            if pd.isnull(row[f'Round_{current_round}_Model_{model_name}']):
                model_response = None
                while True:
                    try:
                        model_response = get_value(model_name, row, current_round)
                        break
                    except Exception as e:
                        print(f"An error happened ({e}). I repeat it.")
                # print(model_response)
                df.at[index, f'Round_{current_round}_Model_{model_name}'] = model_response
                df.to_csv(f'/home/jovyan/work/Round_{current_round}.csv', index=False)  # Save DataFrame to CSV after each row